In [11]:
import torch
import torchvision
from torch.utils.data import DataLoader

import os
import cv2
from tqdm.notebook import trange

import gym
from gym import spaces

from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.dqn.policies import CnnPolicy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import sklearn as sk
from sklearn import datasets
# from stable_baselines3.common.policies import register_policy
# register_policy("myPolicy", CnnPolicy)
from glob import glob

In [12]:
os.chdir('/vol1/idrl/idrl_env/mmd/idrl_mmd/')
norm_models = sorted(glob('*norm*.zip'))
models = sorted(glob('*.zip'))
models = [model for model in models if model not in norm_models]

In [6]:
from sklearn.datasets import load_svmlight_file
from dqn import *
import numpy as np
from stable_baselines3 import DQN


train_norm = load_svmlight_file('data/mnist_data')
x_norm, y_norm = train
x_norm = x_norm.todense()

sortind = np.argsort(y_norm)
x_norm = x_norm[sortind, :]
y_norm = y_norm[sortind]

test_norm = load_svmlight_file('data/mnist_data.t')
testx_norm, testy_norm = test_norm
testx_norm = testx_norm.todense()

In [7]:
width, height = 64, 64
for model_name in norm_models:
    print(model_name)
    model = DQN.load(model_name)
    
    total_rewards = 0
    pred_y_test = []
    count = 0

    for idx in trange(len(testx_norm)):
        # Generate an evaluation observation frame.
        obs = cv2.resize(np.array(testx_norm[idx]).astype(np.float32), (width, height), interpolation = cv2.INTER_CUBIC)
        obs = obs.reshape(width, height, 1)

        # Predict an action based on the observation.
        action, _states = model.predict(obs)
        pred_y_test.append(action)


        # Score the prediction.
        if (action.size > 1):
            # print(action)
            count += 1
            pass
        else:
            reward = 1 if action == testy_norm[idx] else 0
            total_rewards += reward

    print('Accuracy: {:.2f}%'.format(total_rewards / (len(testy_norm) - count) * 100.0))

dqn_cnn_mnist_norm_100_tarun.zip


  0%|          | 0/10000 [00:00<?, ?it/s]

Accuracy: 66.75%
dqn_cnn_mnist_norm_10_tarun.zip


  0%|          | 0/10000 [00:00<?, ?it/s]

Accuracy: 54.57%
dqn_cnn_mnist_norm_1_tarun.zip


...

Accuracy: 62.63%
dqn_cnn_mnist_norm_5_tarun.zip


  0%|          | 0/10000 [00:00<?, ?it/s]

Accuracy: 47.32%
dqn_cnn_mnist_norm_75_tarun.zip


  0%|          | 0/10000 [00:00<?, ?it/s]

Accuracy: 67.92%


In [10]:
protos = sorted(glob('protos/*'))
critics = sorted(glob('critics/*'))
combined = sorted(glob('combined/*'))

epochs = 25

for testm in protos:
#     print(testm)
    selected = np.load(testm)
#     print(selected)
    env = DummyVecEnv([lambda: USPSGym(width=64, height=64, channels=1, dataset=(x_norm[selected, :], y_norm[selected]))])

    # Grab the observation shape for generating evaluation frames.
    width, height = env.observation_space.shape[0], env.observation_space.shape[1]

    test_model = DQN(CnnPolicy, env, verbose=1)

    test_model.learn(total_timesteps=len(x_norm) * epochs)
    
    test_model.save('models/norm_{}_{}.zip'.format(testm, epochs))

    accuracy = eval(test_model, testx_norm, testy_norm)

    error = 100 - accuracy
    print("=============== m={} error={} ===============".format(testm, error))
#     break

for testm in critics:
#     print(testm)
    selected = np.load(testm)
#     print(selected)
    env = DummyVecEnv([lambda: USPSGym(width=64, height=64, channels=1, dataset=(x_norm[selected, :], y_norm[selected]))])

    # Grab the observation shape for generating evaluation frames.
    width, height = env.observation_space.shape[0], env.observation_space.shape[1]

    test_model = DQN(CnnPolicy, env, verbose=1)

    test_model.learn(total_timesteps=len(x_norm) * epochs)

    test_model.save('models/norm_{}_{}.zip'.format(testm, epochs))

    accuracy = eval(test_model, testx_norm, testy_norm)

    error = 100 - accuracy
    print("=============== m={} error={} ===============".format(testm, error))
#     break

for testm in combined:
#     print(testm)
    selected = np.load(testm)
#     print(selected)
    env = DummyVecEnv([lambda: USPSGym(width=64, height=64, channels=1, dataset=(x_norm[selected, :], y_norm[selected]))])

    # Grab the observation shape for generating evaluation frames.
    width, height = env.observation_space.shape[0], env.observation_space.shape[1]

    test_model = DQN(CnnPolicy, env, verbose=1)

    test_model.learn(total_timesteps=len(x_norm) * epochs)

    test_model.save('models/norm_{}_{}.zip'.format(testm, epochs))

    accuracy = eval(test_model, testx_norm, testy_norm)

    error = 100 - accuracy
    print("=============== m={} error={} ===============".format(testm, error))
#     break

Using cpu device
Wrapping the env in a VecTransposeImage.
----------------------------------
| rollout/            |          |
|    exploration rate | 0.962    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 8651     |
|    time_elapsed     | 0        |
|    total timesteps  | 6000     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0431   |
|    n_updates        | 3651     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.924    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 8762     |
|    time_elapsed     | 1        |
|    total timesteps  | 12000    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.886    |
| time/               |          |
|    episodes         | 12      

----------------------------------
| rollout/            |          |
|    exploration rate | 0.164    |
| time/               |          |
|    episodes         | 88       |
|    fps              | 207      |
|    time_elapsed     | 636      |
|    total timesteps  | 132000   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0237   |
|    n_updates        | 20499    |
----------------------------------


KeyboardInterrupt: 

In [4]:
ranges = [1500, 2000, 3000, 4500, 6000]
ranges += [item*2 for item in ranges]
ranges = sorted(list(set(ranges)))
ranges

[1500, 2000, 3000, 4000, 4500, 6000, 9000, 12000]